In [63]:
from bs4 import BeautifulSoup
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import win32gui, win32con
import base64
import sqlite3
import win32crypt
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [64]:
cd D:\USER\OneDrive\jupyter notebook\網路爬蟲

D:\USER\OneDrive\jupyter notebook\網路爬蟲


In [65]:
def GetString(LocalState):
    with open(LocalState,'r',encoding='utf-8') as f:
        s=json.load(f)['os_crypt']['encrypted_key']
    return s

def pull_the_key(base64_encrypted_key):
    encrypted_key_with_header=base64.b64decode(base64_encrypted_key)
    encrypted_key=encrypted_key_with_header[5:]
    key=win32crypt.CryptUnprotectData(encrypted_key,None,None,None,0)[1]
    return key

def DecryptString(key,data):
    nonce,cipherbytes=data[3:15],data[15:]
    aesgcm=AESGCM(key)
    plainbytes=aesgcm.decrypt(nonce,cipherbytes,None)
    plaintext=plainbytes.decode('utf-8')
    return plaintext

def test_cookies(domain_name):
    LocalState = r'C:\Users\User\AppData\Local\Google\Chrome\User Data\Local State' #密钥文件
    Cookies = r'C:\Users\User\AppData\Local\Google\Chrome\User Data\Default\Network\Cookies' #cookie文件
    
    con = sqlite3.connect(Cookies)
    res_sso = con.execute('select host_key,name,encrypted_value from cookies where host_key like "%sso.teachable.com%"').fetchall()
    res_domain_name = con.execute('select host_key,name,encrypted_value from cookies where host_key like "%teachable%" and (' + 'host_key like "%' + domain_name + '%")').fetchall()
    key = pull_the_key(GetString(LocalState))        
    con.close()
    
#    字典回傳    
#    data={}
#    datas={}
#    for i in res_sso:
#        data[i[1]]=DecryptString(key, i[2])
#    datas['sso.teachable.com']=data
#    data={}
#    for i in res_host_key:
#        data[i[1]]=DecryptString(key, i[2])
#    datas[host_key]=data

#    列表回傳  
    datas=[]
    for i in res_sso:
        datas.append(i[1])
        datas.append(DecryptString(key, i[2]))
    for i in res_domain_name:
        datas.append(i[1])
        datas.append(DecryptString(key, i[2]))
    return datas

domain_name="teciram904-s-school" ##############
cookie_data = test_cookies(domain_name)

In [66]:
driver = webdriver.Chrome('D:/USER/OneDrive/jupyter notebook/網路爬蟲/chromedriver.exe')
driver.maximize_window() #最大化窗口 
driver.implicitly_wait(25) #隱性等待 25 秒

<ipython-input-66-860ad3d5fbde>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:/USER/OneDrive/jupyter notebook/網路爬蟲/chromedriver.exe')


In [67]:
url='https://teciram904-s-school.teachable.com/'   
driver.get(url)

for i in range(0,int(len(cookie_data)),2):
    driver.add_cookie({'name': cookie_data[i],'value':cookie_data[i+1]})
url='https://teciram904-s-school.teachable.com/courses'   
driver.get(url)
driver.implicitly_wait(25)

#driver.find_elements_by_css_selector('a.btn.btn-primary.btn-admin')[0].click()
#username='lajadov141@pyrelle.com'
#password='789456123'
#elem = driver.find_element_by_id("email")
#elem.send_keys(username)
#elem = driver.find_element_by_id("password")
#elem.send_keys(password)
#driver.find_elements_by_css_selector('input.btn-primary.button.p-v-2-xs.p-h-5-xs.m-v-3-xs.col-12-xs')[0].click()

In [68]:
elem = driver.find_elements(By.ID,"search-courses")[0]
# 清除搜尋框
elem.clear()
# 輸入課程名稱
product_name="teciram904"
elem.send_keys(product_name)
# 點擊在第一個位置上的課程
driver.find_elements(By.ID,'search-course-button')[0].click()

In [69]:
if driver.find_elements(By.CSS_SELECTOR,"div.course-listing-title")[0].text == product_name:
    driver.find_elements(By.CSS_SELECTOR,"div.course-listing-title")[0].click()
elif driver.find_elements(By.CSS_SELECTOR,"div.course-listing-title")[1].text == product_name:
    driver.find_elements(By.CSS_SELECTOR,"div.course-listing-title")[1].click()
elif driver.find_elements(By.CSS_SELECTOR,"div.course-listing-title")[2].text == product_name:
    driver.find_elements(By.CSS_SELECTOR,"div.course-listing-title")[2].click()
    
driver.find_elements(By.CSS_SELECTOR,"a.edit-button.m-r-3-xs.p-h-3-xs.p-v-2-xs")[0].click()    

In [70]:
block = 'banner'.lower()
blocks = driver.find_elements(By.CSS_SELECTOR,"button.BlockItem-modules__blockName-1-0c6.text-left-xs._3x1ps") 
for index, value in enumerate(blocks):
    if value.text.lower() == block:
        driver.find_elements(By.CSS_SELECTOR,"button.BlockItem-modules__blockName-1-0c6.text-left-xs._3x1ps")[index].click()    
        break

In [71]:
# 找到要讓鼠標移動 (懸停) 到的位置
find_element_hover = driver.find_elements(By.CSS_SELECTOR,"div.MediaUploader-modules__overlay-1UgbV.MediaUploader-modules__overlayWithImage-SwgD3")[0]
# 讓鼠標執行 懸停
ActionChains(driver).move_to_element(find_element_hover).perform()
# 讓 Selenium 點擊 綠色Replace Image 的按鈕
driver.find_elements(By.CSS_SELECTOR,"button._2po1U.MediaUploader-modules__button-3ZLXW.dsp-flex-xs.flex-justify-content-center-xs.flex-align-items-center-xs")[0].click()

In [72]:
driver.switch_to.frame(driver.find_elements(By.XPATH,'//*[@id="filepicker_dialog"]')[0])

In [73]:
driver.find_elements(By.CSS_SELECTOR,"button.btn.btn--primary.ng-binding")[0].click()
time.sleep(1)

In [74]:
# 一級頂層視窗 (檔案選取的對話框)
dialog = win32gui.FindWindow("#32770", "開啟")

# 二級視窗
comboBoxEx32 = win32gui.FindWindowEx(dialog, 0, "ComboBoxEx32", None)

# 三級視窗
comboBox = win32gui.FindWindowEx(comboBoxEx32, 0, "ComboBox", None)

# 四級視窗 -- 檔案路徑輸入區域
edit = win32gui.FindWindowEx(comboBox, 0, "Edit", None)

# 二級視窗 -- 開啟按鈕
button = win32gui.FindWindowEx(dialog, 0, "Button", None)

# 1、輸入檔案路徑
filepath = f"D:\\pexels-nur-andi-ravsanjani-gusma-2422583.jpg"

win32gui.SendMessage(edit, win32con.WM_SETTEXT, None, filepath)

# 2、點選開啟按鈕
win32gui.SendMessage(dialog, win32con.WM_COMMAND, 1, button)

0

In [75]:
driver.switch_to.default_content()
time.sleep(1)
driver.switch_to.frame(driver.find_elements(By.XPATH,'//*[@id="filepicker_dialog"]')[0])
time.sleep(1)
driver.find_elements(By.CSS_SELECTOR,"button.btn.btn--primary.btn--small_screen.ng-binding")[0].click()

In [76]:
time.sleep(5)
driver.find_elements(By.CSS_SELECTOR,"button._2po1U._2po1U.p-v-2-xs.p-h-5-xs._1Q15i._2-Hb2._1yqRw")[0].click()

WebDriverException: Message: target frame detached
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D36903+2517251]
	Ordinal0 [0x00CCF8E1+2095329]
	Ordinal0 [0x00BD2710+1058576]
	Ordinal0 [0x00BC4D4D+1002829]
	Ordinal0 [0x00BC3F8D+999309]
	Ordinal0 [0x00BC4357+1000279]
	Ordinal0 [0x00BDA580+1090944]
	Ordinal0 [0x00BD4A7B+1067643]
	Ordinal0 [0x00BD45F5+1066485]
	Ordinal0 [0x00BD4629+1066537]
	Ordinal0 [0x00BD4DCB+1068491]
	Ordinal0 [0x00BD5070+1069168]
	Ordinal0 [0x00BFD1C2+1233346]
	Ordinal0 [0x00BFD63B+1234491]
	Ordinal0 [0x00C2784C+1407052]
	Ordinal0 [0x00C1650A+1336586]
	Ordinal0 [0x00C25BBF+1399743]
	Ordinal0 [0x00C1639B+1336219]
	Ordinal0 [0x00BF27A7+1189799]
	Ordinal0 [0x00BF3609+1193481]
	GetHandleVerifier [0x00EC5904+1577972]
	GetHandleVerifier [0x00F70B97+2279047]
	GetHandleVerifier [0x00DC6D09+534521]
	GetHandleVerifier [0x00DC5DB9+530601]
	Ordinal0 [0x00CD4FF9+2117625]
	Ordinal0 [0x00CD98A8+2136232]
	Ordinal0 [0x00CD99E2+2136546]
	Ordinal0 [0x00CE